# Setup environment and make sure it works as expected.

This workbook is to validate the environment.

I have built a Docker container based on jupyter/datascience-notebook, with _DifferentialEquations_ and _Plots_ added to the container that I am using locally for these notebooks. Eventually all this should get into this repo, but I am still figureing it out.


In [ ]:
using Pkg
Pkg.status()

Only do the following if you really need to, it can take a long time.

In [ ]:
# Pkg.add("DifferentialEquations")
# Pkg.add("Plots")
# Pkg.status()

In [ ]:
using DifferentialEquations
using Plots

# Excercise 1

_Local extrema of van der Pol oscillation._ In the van der Pol example
(1.2), you can find the local maxima with $max(y,'local')$, and similarly for minima
with $min$; you can find both minima and maxima at once with $minandmax$. Howclose is
the first local minimum value (at $t \approx 1.2$) to its asymptotic value for $t \to \infty$? Likewise for the first local maximum (at $t \approx 3.2$)?

Note, that in Julia, the min/max functions don't work like that, hence the ones I built, at least till I learn of a better way.


## And now an example from the book to see if _I_ understand how to use the package.

Solve: $0.3u'' -(1-u^2)u' + u = 0$

In [ ]:

vdp(du, u, p, t) = ((1-u^2)*du - u) / 0.3
vdpspan = (0.0, 20.0)
vdpprob = SecondOrderODEProblem(vdp, 0.0, 1.0, vdpspan)
vdpsol = solve(vdpprob, reltol=1e-8, abstol=1e-8)

plot(vdpsol, vars=(0, 2))


And get the minimas. First Load my utility routines.

In [ ]:
include("localminmax.jl")

**Note:** I am still trying to see how to increase the accuracy here - the book asks for 6 significant digits, but lowering the tolerance in the solver beyond $10^{-8}$ seems to lower the accuracy. This shouldn't be hard (at least, from the book, it seems trivial in MatLab). That said, it appears that I am only seeing an accuracy of about 4 significant digits, which seems wrong given the tolerance specified.

In [ ]:
sigdigits = 6

result = [v[2] for v in vdpsol.u]
mins = localmins(result)
last_min = nothing
for cur_min in mins
    print("t = ", round(vdpsol.t[cur_min[1]], sigdigits=sigdigits),
          " u = ", round(cur_min[2], sigdigits=sigdigits))
    if last_min != nothing
        println("   Diff to last = ", round(vdpsol.t[cur_min[1]] - last_min, sigdigits=sigdigits))
    else
        println("")
    end
    last_min = vdpsol.t[cur_min[1]]
end
println("")

maxs = localmaxs(result)
last_max = nothing
for cur_max in maxs
    print("t = ", round(vdpsol.t[cur_max[1]], sigdigits=sigdigits),
          " u = ", round(cur_max[2], sigdigits=sigdigits))
    if last_max != nothing
        println("   Diff to last = ", round(vdpsol.t[cur_max[1]] - last_max, sigdigits=sigdigits))
    else
        println("")
    end
    last_max = vdpsol.t[cur_max[1]]
end
println("")


Unfortunately we are not getting the same numbers as in the book, they are the same to 3 significant digits, but the book has 5 and the period is certainly not the same to 5 significant digits.